In [5]:
import googleapiclient.discovery as gapi
from googleapiclient.errors import HttpError
import pandas as pd
import os
import time
import concurrent.futures
import datetime

# Setup

In [6]:
CURRENT_FOLDER = 'C:\\Coding Projects\\YoutubeCookingData\\'

with open(CURRENT_FOLDER + "apiKeys.txt") as f:
    YOUTUBE_API_KEY = f.read()
youtube_service = gapi.build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)

# Final Functions

In [39]:
from datetime import date


def jsonFieldHandler(json, key1, key2='', key3='', key4=''):
    # If the key doesn't exist, return ''
    try:
        if(key1 and key2 and key3 and key4):
            return json[key1][key2][key3][key4]
        elif(key1 and key2 and key3):
            return json[key1][key2][key3]
        elif(key1 and key2):
            return json[key1][key2]
        elif(key1):
            return json[key1]
    except:
        return ''

# Gets the data for all channels and parses it
def getChannelDataHandler(channel_id_list):
    channel_dict_list = []
    channel_response_json = []
    
    # Get all fields for Channel
    for i in range(0, len(channel_id_list), 50):
        channel_response_json += getChannelData(channel_id_list[i:i+50])
        
    # Parse response into dictionary
    for result in channel_response_json:
#         print(f"LOG: Fetching Data for Channel ID {result['id']}")
        channel_dict = {}
        
        channel_dict['Channel ID'] = jsonFieldHandler(result, 'id')
        channel_dict['Title'] = jsonFieldHandler(result, 'snippet','title')
        channel_dict['Description'] = jsonFieldHandler(result, 'snippet','description')
        channel_dict['URL'] = jsonFieldHandler(result, 'snippet','customUrl')
        channel_dict['Channel Created Date'] = jsonFieldHandler(result, 'snippet','publishedAt')
        channel_dict['Thumbnail URL'] = jsonFieldHandler(result, 'snippet', 'thumbnails', 'high', 'url')
        channel_dict['Language'] = jsonFieldHandler(result, 'snippet', 'defaultLanguage')
        channel_dict['Country'] = jsonFieldHandler(result, 'snippet', 'country')
        channel_dict['Views'] = jsonFieldHandler(result, 'statistics', 'viewCount')
        channel_dict['Subscriber Count'] = jsonFieldHandler(result, 'statistics', 'subscriberCount')
        channel_dict['Video Count'] = jsonFieldHandler(result, 'statistics', 'videoCount')
        channel_dict['Topics'] = jsonFieldHandler(result, 'topicDetails', 'topicIds')
        channel_dict['Topic Categories'] = jsonFieldHandler(result, 'topicDetails', 'topicCategories')
        channel_dict['Upload Playlist ID'] = jsonFieldHandler(result, 'contentDetails', 'relatedPlaylists', 'uploads')

        channel_dict_list.append(channel_dict)
        
    return pd.DataFrame(channel_dict_list)

# Gets all fields for a given list of channel
def getChannelData(channel_id_list):
    request = youtube_service.channels().list(
        part=['id', 'snippet', 'statistics', 'topicDetails', 'contentDetails'],
        id=channel_id_list,
        maxResults = 50
    )
    
    results = request.execute()
    return results['items']

# Returns the upload playlist for a list of channel ids
def getUploadPlaylistsforChannelHandler(channel_id_list):
    channel_response_json = []
    upload_playlist_list = []
    
    # Get upload playlists
    for i in range(0, len(channel_id_list), 50):
        channel_response_json += getUploadPlaylistsforChannel(channel_id_list[i:i+50])
    
    # Get only the channel's upload playlist id
    return [jsonFieldHandler(channel_json, 'contentDetails', 'relatedPlaylists', 'uploads') for channel_json in channel_response_json]

# Gets upload playlist ID for a list of channel IDs
def getUploadPlaylistsforChannel(channel_id_list):
    request = youtube_service.channels().list(
    part=['contentDetails'],
        id=channel_id_list,
        maxResults=50
    )
    
    result = request.execute()
    
    return result['items']
    
    
    
# Function that returns all videoIds for a given channel's upload playlist
def getVideoListForPlaylist(upload_playlist_id, page_token=''):
    video_ids = []

    t1 = time.time()
    
    request = youtube_service.playlistItems().list(
        part="contentDetails",
        playlistId=upload_playlist_id,
        maxResults=50,
        pageToken=page_token
    )
    
    try:
        results = request.execute()
        result_videos = results['items']
    except HttpError as err:
        appendToLog(HttpError)
        raise err
    
    # If there is a next page then append videos to result array
    if('nextPageToken' in results):
        nextPage = results['nextPageToken']
        result_videos += getVideoListForPlaylist(upload_playlist_id, nextPage)
    
    # If not called by another instance of this function, return only IDs
    if(page_token == ''):
#         print(f"LOG: Working on Channel ID {channelId}.")
        t2 = time.time()
        appendToLog(f'Time elapsed for videos of playlist {upload_playlist_id} {t2-t1} for {len(result_videos)} videos')
        return [vid['contentDetails']['videoId'] for vid in result_videos]
    else:
        # If recursing, return whole response list
        return result_videos

# Function that parses the return JSON from getVideoListForPlaylist and gets additional columns
def getVideoDataHandler(video_id_list):
    video_response_json = []
    video_dict_list = []
    
    # Get all fields for Video
    for i in range(0, len(video_id_list), 50):
        try:
            video_response_json += getVideoData(video_id_list[i:i+50])
        except HttpError as err:
            raise err
    
    # Parse response into dictionary
    for result in video_response_json:
#         print(f"\tLOG: Fetching Data for Video ID {result['id']}")
        video_dict = {}
        
        video_dict['Title'] = jsonFieldHandler(result, 'snippet', 'title')
        video_dict['Video ID'] = jsonFieldHandler(result, 'id')
        video_dict['Channel ID'] = jsonFieldHandler(result, 'snippet', 'channelId')
        video_dict['Duration'] = jsonFieldHandler(result, 'contentDetails', 'duration')
        video_dict['Description'] = jsonFieldHandler(result, 'snippet', 'description')
        video_dict['Publish Date'] = jsonFieldHandler(result, 'snippet', 'publishedAt')
        video_dict['Thumbnail URL'] = jsonFieldHandler(result, 'snippet', 'thumbnails', 'high',  'url')
        video_dict['View Count'] = jsonFieldHandler(result, 'statistics', 'viewCount')
        video_dict['Like Count'] = jsonFieldHandler(result, 'statistics', 'likeCount')
        video_dict['Comment Count'] = jsonFieldHandler(result, 'statistics', 'commentCount')        
        video_dict['Video Definition'] = jsonFieldHandler(result, 'contentDetails', 'definition')
        video_dict['Default Audio Language'] = jsonFieldHandler(result, 'snippet', 'defaultAudioLanguage')
        video_dict['Tags'] = jsonFieldHandler(result, 'snippet', 'tags')
        video_dict['Category ID'] = jsonFieldHandler(result, 'snippet', 'categoryId')
        video_dict['Topic Details'] = jsonFieldHandler(result, 'topicDetails')
        video_dict['Made for Kids'] = jsonFieldHandler(result, 'status', 'madeForKids')
        video_dict['Favorite Count'] = jsonFieldHandler(result, 'statistics', 'favoriteCount')
        video_dict_list.append(video_dict)
        
    return pd.DataFrame(video_dict_list)
     
def getVideoData(list_of_videos):
    request = youtube_service.videos().list(
        part=['contentDetails', 'liveStreamingDetails', 'id',
              'snippet', 'statistics', 'status', 'topicDetails'],
        id=list_of_videos,
        maxResults=50,
    )
    
    try:
        results = request.execute()
        return results['items']
    except HttpError as err: 
        # If error, return 
        appendToLog(HttpError)
        raise err

# Gets the comment threads for all videos in a given list
def getCommentThreadsHandler(list_of_videos):
    dataframes_list = []
    
    for video_id in list_of_videos:
        try:
            dataframes_list.append(getCommentsForVideo(video_id))
        except HttpError as err:
            # If it is bad request, try again in 5 seconds. Else raise error
            if(err.resp.status == 400):
                try:
                    appendToLog(f"Potentially transient ({err.resp.status}), trying again for video {video_id}")
                    time.sleep(600)
                    dataframes_list.append(getCommentsForVideo(video_id))
                except HttpError as err:
                    raise err
            else:
                raise err

    return pd.concat(dataframes_list, ignore_index=True)

# Get list of comment threads for a given video, returns a dataframe
def getCommentsForVideo(video_id, page_token=""):
#     print(f"\tLOG: Fetching Comment Data for Video ID {video_id}, {page_token}")
    t1 = time.time()
    request = youtube_service.commentThreads().list(
        part=['id', 'replies', 'snippet'],
        videoId=video_id,
        maxResults=100,
        pageToken=page_token
    )

    try:
        results = request.execute()
        result_comments = results['items']
    
        if('nextPageToken' in results):
            next_page = results['nextPageToken']
            result_comments += getCommentsForVideo(video_id, next_page)
        
        # If not recursing, parse the full response
        if(page_token == ''):
            t2 = time.time()
            appendToLog(f'Time elapsed for comments on video {video_id} {t2-t1} for {len(result_comments)} comments')
            return parseCommentThreadResponse(result_comments)
        else: 
            # Return raw results if called by another instance of this function
            return result_comments
    except HttpError as err:
        appendToLog(f'{err.resp.status} - {err._get_reason()} - {video_id} with page {page_token}')
        raise err

# Parse comment threads response JSON
def parseCommentThreadResponse(list_of_threads):
    thread_dict_list = []
    
    for thread in list_of_threads:
#         print(f"\t\tLOG: Fetching Data for Comment Thread ID {thread['id']}")
        thread_dict = {}
        
        thread_dict['Comment Thread ID'] = jsonFieldHandler(thread, 'id')
        thread_dict['Video ID'] = jsonFieldHandler(thread, 'snippet', 'videoId')
        thread_dict['Top Level Comment'] = jsonFieldHandler(thread, 'snippet', 'topLevelComment')
        thread_dict['Total Replies'] = jsonFieldHandler(thread, 'snippet', 'totalReplyCount')
        thread_dict['Can Reply'] = jsonFieldHandler(thread, 'snippet', 'canReply')
        thread_dict['Replies'] = jsonFieldHandler(thread, 'replies')
        
        thread_dict_list.append(thread_dict)
    
    return pd.DataFrame(thread_dict_list)

def appendToLog(message): 
    print(message)
    current_time = datetime.datetime.now()
    
    file_path = CURRENT_FOLDER + '\\Logs\\' + f"Log {current_time.year}-{current_time.month}-{current_time.day}.txt"

    if(os.path.exists(file_path)):
        append_write = 'a'
    else:
        append_write ='w'
    
    with open(file_path, append_write) as f:
        f.write(message + '\n')

def main(channels_flag=True, videos_flag=True, comments_flag=True, current_folder=''):
    start_time = time.time()
    appendToLog(f"Starting execution for channel data: {channels_flag}, video data: {videos_flag}, comment data: {comments_flag}")
    CHANNEL_ID_TOTAL_DF = pd.read_csv(current_folder + "Channel IDs.csv")

    # File Paths
    channels_file_path = current_folder + "Channel Data.csv"
    videos_file_path = current_folder + "Video Data.csv"
    comments_file_path = current_folder + "Comment Data Raw.csv"
    comment_progress_file_path = current_folder + "Comment Progress.csv"
    
    # If Channels step is done, skip
    if(channels_flag):
        # If the file exists check how many channels still need to be queried, else get the whole list
        if(os.path.exists(channels_file_path)):
            # Get list of channel ids that have been queried and compare against the total list of ids
            channel_data_done_ids = pd.read_csv(channels_file_path)['Channel ID'].to_list()
            
            channelIdsToFetch = CHANNEL_ID_TOTAL_DF[~CHANNEL_ID_TOTAL_DF['ID'].isin(channel_data_done_ids)]['ID'].to_list()
            appendToLog(f"{len(channel_data_done_ids)} channels already done out of {CHANNEL_ID_TOTAL_DF['ID'].shape[0]}")
        else:
            channelIdsToFetch = CHANNEL_ID_TOTAL_DF['ID'].to_list()
            appendToLog(f"No channels already done, querying all {len(channelIdsToFetch)} IDs")
        
        # If there are channels to fetch, get data and write to csv
        if(len(channelIdsToFetch) > 0):
            channels_df = getChannelDataHandler(channelIdsToFetch)
            # If there is an existing file, concat to avoid overwriting
            if(os.path.exists(channels_file_path)):
                    channels_df = pd.concat([channels_df, pd.read_csv(channels_file_path)], ignore_index=True)
            channels_df.to_csv(channels_file_path, index=False)
            appendToLog(f"Done {channels_df.shape[0]} channels")
    
    # Check video progress, skip if done
    if(videos_flag):
        video_ids = []
        videos_list = []
        # Check if a given channel has already had its videos fetched
        if(os.path.exists(videos_file_path)):
            channel_data_done_ids_videos = pd.read_csv(videos_file_path)['Channel ID'].unique().tolist()
            
            channel_ids_to_fetch_videos = CHANNEL_ID_TOTAL_DF[~CHANNEL_ID_TOTAL_DF['ID'].isin(channel_data_done_ids_videos)]['ID'].to_list()
            playlist_ids_to_fetch_videos = getUploadPlaylistsforChannelHandler(channel_ids_to_fetch_videos)
            appendToLog(f"{len(channel_data_done_ids_videos)} channel's videos already done out of {len(CHANNEL_ID_TOTAL_DF['ID'].to_list())}")
            # Read in current data
            videos_list.append(pd.read_csv(videos_file_path))
        else:
            channel_ids_to_fetch_videos = CHANNEL_ID_TOTAL_DF['ID'].to_list()
            appendToLog(f"No channel's videos already done, querying all {len(channel_ids_to_fetch_videos)} channels")
            playlist_ids_to_fetch_videos = getUploadPlaylistsforChannelHandler(channel_ids_to_fetch_videos)
        for playlist_id in playlist_ids_to_fetch_videos:
            try:
                video_ids = getVideoListForPlaylist(playlist_id)
                videos_list.append(getVideoDataHandler(video_ids))
            except HttpError as err:
                appendToLog(f"Error caught while fetching video data, ending execution")
                # Write current data to file
                if(os.path.exists(videos_file_path)):
                    videos_list += pd.read_csv(videos_file_path)
                pd.concat(videos_list, ignore_index=True).to_csv(videos_file_path, index=False)
                return  

        # If all videos are read successfully, write to file and continue
        if(len(channel_ids_to_fetch_videos) > 0):
            if(os.path.exists(videos_file_path)):
                        videos_list += pd.read_csv(videos_file_path)
            pd.concat(videos_list, ignore_index=True).to_csv(videos_file_path, index=False)
            
    if(comments_flag):
        comments_list = []
        if(os.path.exists(videos_file_path)):
            videos_data_total_ids_df = pd.read_csv(videos_file_path)
            if(os.path.exists(comment_progress_file_path)):
                videos_data_done_ids_comments = pd.read_csv(comment_progress_file_path)['Video ID'].tolist()
                # Only fetch videos that have comments
                video_ids_to_fetch_comments = videos_data_total_ids_df[~videos_data_total_ids_df['Video ID'].isin(videos_data_done_ids_comments) 
                & videos_data_total_ids_df['Comment Count'] > 0]['Video ID'].to_list()
                appendToLog(f"Done {len(videos_data_done_ids_comments)} video's comments out of {videos_data_total_ids_df.shape[0]}")
                # Read in current data
                comments_list.append(pd.read_csv(comments_file_path))
            else:
                # Only fetch videos that have comments
                video_ids_to_fetch_comments = videos_data_total_ids_df[videos_data_total_ids_df['Comment Count'] > 0]['Video ID'].to_list()
                appendToLog(f"No comments done, getting data for all {len(video_ids_to_fetch_comments)} videos")
        else:
            appendToLog("No video data file, exiting")
            return
        for video_id in video_ids_to_fetch_comments:
            try:
                comments_list.append(getCommentThreadsHandler([video_id]))
            except HttpError as err:
                appendToLog(f"Error caught while fetching comment data, ending execution")
                # Write current data to file 
                if(os.path.exists(comments_file_path)):
                    comments_list += pd.read_csv(comments_file_path)
                pd.concat(comments_list, ignore_index=True).to_csv(comments_file_path,index=False)
                return
        # Write each video's comments to the file if not excepted
        if(len(video_ids_to_fetch_comments) > 0):
            if(os.path.exists(comments_file_path)):
                comments_list += pd.read_csv(comments_file_path)
            pd.concat(comments_list, ignore_index=True).to_csv(comments_file_path, index=False)
    end_time = time.time()
    appendToLog(f"Finished data gathering. Overall execution time: {end_time - start_time}")
        
    return
    
# Takes bulk comment data and splits into subsheets, one for each channel
def postProcessingComments(filePath, targetPath):
    comment_data = pd.read_csv(filePath)
    appendToLog(f"Beginning comment post processing for {comment_data.shape[0]} comment threads.")
    videos_channels = pd.read_csv("Video Data.csv")
    video_channels = videos_channels[["Video ID", "Channel ID"]]
    done_videos_df = pd.DataFrame()

    for channelId in video_channels['Channel ID'].unique():
        temp_df = comment_data[comment_data['Video ID'].isin(video_channels[video_channels['Channel ID'] == channelId]['Video ID'])]

        # If there was more than one comment thread found for the channel
        if(temp_df.shape[0] > 1):
            file_path = f"{targetPath}{channelId}_comment_data.csv"
            # If the file exists, then read the current data drop duplicates and save with the new data. Else just create the file
            if(os.path.exists(file_path)):
                pd.concat([temp_df, pd.read_csv(file_path)], ignore_index=True).drop_duplicates().to_csv(file_path, index=False)
            else:
                temp_df.drop_duplicates().to_csv(file_path, index=False)
            appendToLog(f"Saved comment data for {channelId} in {file_path}")
            done_videos_df = pd.concat([done_videos_df, temp_df[['Video ID']]], ignore_index=True)
            
    
    # Save videos that have been done to csv file for later reference
    if(os.path.exists("Comment Progress.csv")):
        pd.concat([done_videos_df, pd.read_csv("Comment Progress.csv")], ignore_index=True).to_csv("Comment Progress.csv", index=False)
    else:
        done_videos_df.to_csv("Comment Progress.csv", index=False)
        

    

In [8]:
main(True, True, True, current_folder=CURRENT_FOLDER)

Starting execution for channel data: True, video data: True, comment data: True
No channels already done, querying all 60 IDs
Done 60 channels
No channel's videos already done, querying all 60 channels
Time elapsed for videos of playlist UUbRj3Tcy1Zoz3rcf83nW5kw 3.6363165378570557 for 1583 videos
Time elapsed for videos of playlist UU-N2GFf6Dmna8NZ51vO--Lg 2.4539260864257812 for 1005 videos
Time elapsed for videos of playlist UUsP7Bpw36J666Fct5M8u-ZA 0.9380927085876465 for 494 videos
Time elapsed for videos of playlist UURPWH2YmwgbVGz6zJZG1afA 0.9843504428863525 for 500 videos
Time elapsed for videos of playlist UU58sWqKKNBQDphes0Ys037w 0.29790329933166504 for 106 videos
Time elapsed for videos of playlist UU1aJuxLHlw8bBV6mfCqVfog 2.479327917098999 for 1059 videos
Time elapsed for videos of playlist UUsWpnu6EwIYDvlHoOESpwYg 2.524259567260742 for 1100 videos
Time elapsed for videos of playlist UUcjhYlL1WRBjKaJsMH_h7Lg 3.204202651977539 for 1352 videos
Time elapsed for videos of playlist

In [40]:
postProcessingComments("Comment Data Raw.csv", CURRENT_FOLDER + "Comment Data\\")

Beginning comment post processing for 713311 comment threads.
Saved comment data for UCbRj3Tcy1Zoz3rcf83nW5kw in C:\Coding Projects\YoutubeCookingData\Comment Data\UCbRj3Tcy1Zoz3rcf83nW5kw_comment_data.csv


# Data Validation

In [37]:
pd.read_csv("Comment Data Raw.csv").shape, pd.read_csv("Comment Data\\UCbRj3Tcy1Zoz3rcf83nW5kw_comment_data.csv").shape, pd.read_csv("Comment Progress.csv").shape

((713311, 6), (713311, 6), (713311, 1))

In [38]:
data = pd.read_csv("Comment Progress.csv")
data

,Video ID
0,Oi65_EtsBBY
1,Oi65_EtsBBY
2,Oi65_EtsBBY
3,Oi65_EtsBBY
4,Oi65_EtsBBY
...,...
713306,AvwtESN530E
713307,AvwtESN530E
713308,AvwtESN530E
713309,AvwtESN530E


In [33]:
pd.read_csv("Video Data.csv")

,Title,Video ID,Channel ID,Duration,Description,Publish Date,Thumbnail URL,View Count,Like Count,Comment Count,Video Definition,Default Audio Language,Tags,Category ID,Topic Details,Made for Kids,Favorite Count
0,FRENCH ONION MAC & CHEESE (ONE OF THE BEST THI...,Oi65_EtsBBY,UCbRj3Tcy1Zoz3rcf83nW5kw,PT8M32S,It's that time of year when a warm bowl of che...,2022-11-04T18:29:42Z,https://i.ytimg.com/vi/Oi65_EtsBBY/hqdefault.jpg,96067.0,6763.0,427.0,hd,en,"['sam the cooking guy', 'cooking', 'sam cookin...",24,{'topicCategories': ['https://en.wikipedia.org...,False,0
1,MY NEW FAVORITE BREAKFAST CREATION... | SAM TH...,dDGj3M8MWFI,UCbRj3Tcy1Zoz3rcf83nW5kw,PT13M24S,Save 33% on your first Native Plastic-Free Deo...,2022-11-02T20:53:00Z,https://i.ytimg.com/vi/dDGj3M8MWFI/hqdefault.jpg,165154.0,6057.0,363.0,hd,en,"['sam the cooking guy', 'cooking', 'sam cookin...",24,{'topicCategories': ['https://en.wikipedia.org...,False,0
2,THE BEST SURF & TURF BURGER...WOW! | SAM THE C...,xeZ--suEw5Y,UCbRj3Tcy1Zoz3rcf83nW5kw,PT10M10S,Surf & Turf is just one of those amazing thing...,2022-10-28T18:12:24Z,https://i.ytimg.com/vi/xeZ--suEw5Y/hqdefault.jpg,152488.0,7913.0,468.0,hd,en,"['sam the cooking guy', 'cooking', 'sam cookin...",24,{'topicCategories': ['https://en.wikipedia.org...,False,0
3,AN ABSOLUTELY PERFECT CALIFORNIA BURRITO AT HO...,a_x4QWrp_xs,UCbRj3Tcy1Zoz3rcf83nW5kw,PT14M8S,It's been 4 long years since we last made a ve...,2022-10-26T15:53:04Z,https://i.ytimg.com/vi/a_x4QWrp_xs/hqdefault.jpg,364312.0,13604.0,790.0,hd,en,"['sam the cooking guy', 'cooking', 'sam cookin...",24,{'topicCategories': ['https://en.wikipedia.org...,False,0
4,WE'RE TURNING PASTA CARBONARA INTO PIZZA! | SA...,hOGck-w0_0w,UCbRj3Tcy1Zoz3rcf83nW5kw,PT10M9S,"It's simple - you take something you love, lik...",2022-10-21T20:18:24Z,https://i.ytimg.com/vi/hOGck-w0_0w/hqdefault.jpg,164263.0,7106.0,713.0,hd,en,"['sam the cooking guy', 'cooking', 'sam cookin...",24,{'topicCategories': ['https://en.wikipedia.org...,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58329,Xiao Ke Ai - Feichang Fresh (MV) 小可爱 非常fresh组合,lsAqX1nhMF8,UCyEA3vUnlpg0xzkECEq1rOA,PT3M52S,China's hottest Laowai boyband strikes once ag...,2011-02-08T12:34:48Z,https://i.ytimg.com/vi/lsAqX1nhMF8/hqdefault.jpg,32978.0,216.0,26.0,hd,NaN,"['feichang', 'fresh', '非常', '老外', '唱歌', '中文', ...",10,{'topicCategories': ['https://en.wikipedia.org...,False,0
58330,Happy Chinese New Year! 兔年快乐! 恭喜发财! 新年快乐! Chin...,yC6nMRhKF4o,UCyEA3vUnlpg0xzkECEq1rOA,PT3M5S,Happy Chinese New Year!\r\n\r\nSongs we used i...,2010-12-20T12:12:36Z,https://i.ytimg.com/vi/yC6nMRhKF4o/hqdefault.jpg,12203.0,131.0,16.0,hd,NaN,"['feichang', 'fresh', 'chinese', 'classics', '...",10,{'topicCategories': ['https://en.wikipedia.org...,False,0
58331,"Mei Nü, Guo Lai (Shorty, Come Here) - Feichang...",OlH1AgR8ZPE,UCyEA3vUnlpg0xzkECEq1rOA,PT5M2S,The Feichang Crew returns with an all new all-...,2010-12-15T11:10:14Z,https://i.ytimg.com/vi/OlH1AgR8ZPE/hqdefault.jpg,33031.0,292.0,49.0,hd,NaN,"['老外', '中文', '说唱', '唱歌', '美女', '过来', 'Feichang...",10,{'topicCategories': ['https://en.wikipedia.org...,False,0
58332,A Beijing Love Song - Feichang Fresh / 非常 FRE...,2TvzKEG1N0M,UCyEA3vUnlpg0xzkECEq1rOA,PT3M28S,A love song for one of the greatest cities in ...,2010-11-29T03:00:19Z,https://i.ytimg.com/vi/2TvzKEG1N0M/hqdefault.jpg,62082.0,645.0,102.0,hd,NaN,"['gei beijing de qing ge', 'bei jing', 'foreig...",10,{'topicCategories': ['https://en.wikipedia.org...,False,0
